In [9]:
import numpy as np
from keras.models import load_model # 저장된 모델을 로드하기 위함
from keras.preprocessing import image
import matplotlib.pyplot as plt

model = load_model('kvasir_v2.h5')
print(model.summary())

img_path = 'e0fe569d-cc8d-4871-88d0-5477f4aa6e66.jpg'
img = image.load_img(img_path, target_size=(128, 128))
img_tensor = image.img_to_array(img)
img_tensor = np.expand_dims(img_tensor, axis=0)
img_tensor /= 255.

OSError: Unable to open file (truncated file: eof = 8388608, sblock->base_addr = 0, stored_eof = 88852744)

In [4]:
#특정 입력영상에 대한 각 레이어의 Activation Map 출력
from keras import models

layer_outputs = [layer.output # for loop을 통해서 총 8개의 output이 들어가게 된다
                 for layer in model.layers[:8]]

activation_model = models.Model(inputs = model.input,
                                outputs = layer_outputs)

activations = activation_model.predict(img_tensor)

layer_names = []              # 층의 이름을 그래프 제목으로 사용, 모델에 있는 layer의 이름을 가져온다
for layer in model.layers[:8]:
    layer_names.append(layer.name)

images_per_row = 20

NameError: name 'model' is not defined

In [ ]:
# 프로젝트 시 이 셀과 밑의 셀 반드시 포함하여 filter에 관한 내용을 분석하여야 한다
# feature map

for layer_name, layer_activation in zip(layer_names, activations): # zip:두 인자들을 하나의 짝으로 묶어준다/연결시켜준다

    n_features = layer_activation.shape[-1] # layer의 feature의 개수를 추출하여 n_features에 저장
    size = layer_activation.shape[1]        # 영상의 x,y사이즈가 동일해서 그 중 하나를 가져온 것


    n_cols = n_features // images_per_row
    # images_per_row = 20이므로 나눈 몫이 열의 개수가 된다
    display_grid = np.zeros((size * n_cols, images_per_row * size))
    # 영상의 픽셀 사이즈를 정함. 앞: 가로축 길이 / 뒤: 세로축의 길이
    # 즉, 전체 디스플레이할 크기를 의미한다


    for col in range(n_cols):
        for row in range(images_per_row):
            channel_image = layer_activation[0, :, :, col * images_per_row + row] # for loop으로 하나씩 feature를 추출하는
            
            # 영상의 intensity를 조정하는 과정 (Normalize하는 과정)
            channel_image -= channel_image.mean()
            channel_image /= channel_image.std()
            channel_image *= 64  # 전체 영상의 variation을 일정하게 잡아주도록 64를 곱해준다. 일종의 histogram equalization과 비슷
            channel_image += 128 # 영상의 평균값이 128이 되도록 만들어 준다
            
            channel_image = np.clip(channel_image, 0, 255).astype('uint8')
            display_grid[col * size : (col + 1) * size, # 처리된 영상을 각 그리드에 뿌려준다
                         row * size : (row + 1) * size] = channel_image


    scale = 1. / size
    plt.figure(figsize=(scale * display_grid.shape[1], scale * display_grid.shape[0]))
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='viridis')

plt.show() # 하나의 layer가 완성되면 for loop이 멈추고 plt.show()로 출력된다. 즉, layer하나하나씩 출력된다

In [ ]:
#각 레이어의 필터 특성 출력

from keras.applications import VGG16
from keras import backend as K

model = VGG16(weights='imagenet',
              include_top=False)

In [ ]:
  # 1번째 함수
def deprocess_image(x):
    x -= x.mean()
    x /= (x.std() + 1e-5)
    x *= 0.1     # standard deviation의 분포를 조정한다. mean과 표준편차를 변화시켜준다
    x += 0.5
    x = np.clip(x, 0, 1)
    # 즉, 데이터의 분포를 내가 원하는대로 조정하는 과정이다

    x *= 255 # 표준편차를 255로 맞춰줌
    x = np.clip(x, 0, 255).astype('uint8')
    return x
  
# 2번째 함수
# 영상의 필터를 잡아주는 역할  
def generate_pattern(layer_name, filter_index, size=128):
    layer_output = model.get_layer(layer_name).output  # get_layer: 해당 모델의 layer를 가져온다
    loss = K.mean(layer_output[:, :, :, filter_index]) # 가져온 layer에 해당하는 출력을 지정한다

    grads = K.gradients(loss, model.input)[0]          # modle.input에 대한 loss의 gradient를 출력해준다. 즉, gradient를 가져오는 과정이다
    grads /= (K.sqrt(K.mean(K.square(grads))) + 1e-5)  # 전체의 gradient값으로 normalize 해주는 과정
    iterate = K.function([model.input], [loss, grads]) # loss와 gradient를 출력해주는 함수 생성
    
    input_img_data = np.random.random((1, size, size, 3)) * 20 + 128.
    # 128을 기준으로 랜덤값을 집어넣는다(0에서 1사이 float를 출력). 여기서는 거기다가 20을 곱해줌
    # 128(회색). 랜덤영상을 만들어주는 과정

    step = 1.
    for i in range(40):
        loss_value, grads_value = iterate([input_img_data])
        input_img_data += grads_value * step
        
    img = input_img_data[0]
    return deprocess_image(img)
  
for layer_name in ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1']: # VCG16에 있는 레이어의 이름들
    size = 64       # 영상 사이즈
    margin = 5

    results = np.zeros((8 * size + 7 * margin, 8 * size + 7 * margin, 3), dtype='uint8')

    for i in range(8):   # filter의 개수를 지정해줌 --> 총 64개
        for j in range(8): 

            filter_img = generate_pattern(layer_name, i + (j * 8), size=size) # layer name으로부터 각 layer의 pattern을 가져온다
            # 각 filter의 특징을 영상으로 저장하는 것이다

            horizontal_start = i * size + i * margin
            horizontal_end = horizontal_start + size
            vertical_start = j * size + j * margin
            vertical_end = vertical_start + size
            results[horizontal_start: horizontal_end, vertical_start: vertical_end, :] = filter_img


    plt.figure(figsize=(20, 20))
    plt.imshow(results)
    plt.show()

In [ ]:
# 여기부터 매우매우 중요함!
from keras.applications.vgg16 import preprocess_input, decode_predictions

K.clear_session()
model = VGG16(weights='imagenet')

In [ ]:
img_path = 'e0fe569d-cc8d-4871-88d0-5477f4aa6e66.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

In [ ]:
preds = model.predict(x)
print('Predicted:', decode_predictions(preds, top=3)[0]) # 제일 처음 3개를 가져와서 출력한다

In [ ]:
# 여기부터 셀 매우 중요 --> 응용해서 쓸 수 있어야 한다

predict_output = model.output[:, np.argmax(preds[0])]

last_conv_layer = model.get_layer('block5_conv3') # 마지막 층
grads = K.gradients(predict_output, last_conv_layer.output)[0] 

pooled_grads = K.mean(grads, axis=(0, 1, 2)) # 전체의 average가 가장 높은 gradient를 갖는 필터 하나를 선택한다

iterate = K.function([model.input], [pooled_grads, last_conv_layer.output[0]]) # 마지막 레이어를 가져온다
pooled_grads_value, conv_layer_output_value = iterate([x])

In [ ]:
for i in range(512):
    conv_layer_output_value[:, :, i] *= pooled_grads_value[i] # 출력 중 가장 gradient가 높은 것들을 곱한다
    heatmap = np.mean(conv_layer_output_value, axis=-1) # 마지막 axis가 feature에 해당되는 axis
    
heatmap = np.maximum(heatmap, 0)
heatmap /= np.max(heatmap)
plt.matshow(heatmap) # heatmap
plt.show()

plt.matshow(img) # 원본 영상
plt.show()
# heatmap: convolution layer에서 가장 값이 높은 것을 표시한다

In [ ]:
import cv2

background=image.img_to_array(img)
heatmap = cv2.resize(heatmap, (background.shape[1], background.shape[0])) # 영상 사이즈를 키운다
heatmap = np.uint8(255 * heatmap)                      # 전체 범위를 255로 키운다
heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET) # 컬러맵을 바꿔줌

superimposed_img =  np.uint8(heatmap * 0.4+ background*0.6) # 영상 mixing. 두 비율의 합이 1이 되도록 설정해주면 된다 (0.4 + 0.6 = 1)
plt.matshow(superimposed_img)
plt.show()

cv2.imwrite('res_img.jpg', superimposed_img)